In [22]:
!pip install wptools

In [23]:
!pip install deep-translator

In [24]:
!pip install beautifulsoup4

## Mount Google Drive

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [53]:
%cd 'drive/MyDrive/Thesis-MSc/wikipedia-scraping/'
# %cd 'Thesis-MSc/wikipedia-scraping/'

/content/drive/MyDrive/Thesis-MSc/wikipedia-scraping


In [50]:
%cd drive
!ls

/content/drive
MyDrive


In [51]:
%cd MyDrive

/content/drive/MyDrive


In [52]:
!ls

 Advanced-ML				     'LLMs Notes'
'AI in Audit.gdoc'			     'Medical Demand Project.gdoc'
'Arabic Language Models.gdoc'		     'Meeting Minutes.gdoc'
'Arabic LMs.gdoc'			     'Outline - Hiyam.gdoc'
 CMPS207				      Pretrm-Birth-Data
'Colab Notebooks'			      Sectarian-Network-Detection
 Dementia				      Thesis-MSc
'Events Timeline.gsheet'		     'Web Hosting Options.gdoc'
 Generalized-Network-Dismantling-References


In [25]:
import requests
import wptools
import re
import pandas as pd
import os
import csv
import json
from deep_translator import GoogleTranslator
import urllib3
from bs4 import BeautifulSoup
import string

In [26]:
page = wptools.page('1982_Lebanon_War')
page.get_parse()

en.wikipedia.org (parse) 1982_Lebanon_War
1982 Lebanon War (en) data
{
  image: <list(0)> 
  infobox: <dict(18)> conflict, partof, image, caption, date, plac...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:1...
  pageid: 181820
  parsetree: <str(172163)> <root><template><title>short descriptio...
  requests: <list(1)> parse
  title: 1982 Lebanon War
  wikibase: Q49103
  wikidata_url: https://www.wikidata.org/wiki/Q49103
  wikitext: <str(133543)> {{short description|Fighting between Isr...
}


In [27]:
session = requests.Session()

url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "titles": "1982_Lebanon_War",
    "prop": "links",
    "pllimit": "max"
}

page = wptools.page('1982 Lebanon War')

response = session.get(url=url, params=params)
data = response.json()
pages = data["query"]["pages"]

pg_count = 1
page_titles = []

print("Page %d" % pg_count)
for key, val in pages.items():
    for link in val["links"]:
        print(link["title"])
        page_titles.append(link["title"])

while "continue" in data:
    plcontinue = data["continue"]["plcontinue"]
    params["plcontinue"] = plcontinue

    response = session.get(url=url, params=params)
    data = response.json()
    pages = data["query"]["pages"]

    pg_count += 1

    print("\nPage %d" % pg_count)
    for key, val in pages.items():
        for link in val["links"]:
            print(link["title"])
            page_titles.append(link["title"])

print("%d titles found." % len(page_titles))

Page 1
162nd Division (IDF)
17 May Agreement
1948 Arab-Israeli war
1948 Arab–Israeli War
1948 Palestine war
1949 Armistice Agreements
1968 Israeli raid on Lebanon
1978 South Lebanon conflict
1981 Iraqi embassy bombing in Beirut
1982 (2019 film)
1982 Beirut bombing
1982 kidnapping of Iranian diplomats
1983 Beirut barracks bombings
1983 US embassy bombing in Beirut
1983 United States Embassy bombing
1984 Sohmor massacre
1984 United States embassy annex bombing in Beirut
1985 Beirut car bombings
1991 Iraqi missile attacks against Israel
1st Armoured Division (Syria)
2000 Camp David Summit
2002 Arab League summit
2004 United States presidential election
2006 Gaza–Israel conflict
2006 Israeli operation in Beit Hanoun
2006 Lebanon War
2010 Palestinian militancy campaign
2010–2011 Israeli–Palestinian peace talks
2011 southern Israel cross-border attacks
2012 Israeli operation in the Gaza Strip
2013–2014 Israeli–Palestinian peace talks
2014 Gaza War
2015–2016 wave of violence in the Israeli–Pa

In [28]:
def get_arabic_name(english_phrase):
    # if we remove all numbers from a string and the string remains empty - then we won't translate that
    english_phrase_mod = ''.join([i for i in english_phrase if not i.isdigit()])
    english_phrase_mod = english_phrase_mod.translate(str.maketrans('', '', string.punctuation))
    english_phrase_mod = english_phrase_mod.strip()
    if english_phrase_mod == '':
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1
    else:
        english_phrase = english_phrase_mod

    if GoogleTranslator(source='auto', target='ar').translate(english_phrase) == english_phrase:
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1

    response = requests.get('http://en.wikipedia.org/wiki/{}'.format(english_phrase))
    if response.status_code != 200:
        print('Cannot find the link: {}'.format('http://en.wikipedia.org/wiki/{}'.format(english_phrase)))
        return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

    http = urllib3.PoolManager()
    url = 'http://en.wikipedia.org/wiki/{}'.format(english_phrase)
    response = http.request('GET', url)

    # get languages and links
    soup = BeautifulSoup(response.data)
    links = [(el.get('lang'), el.get('href')) for el in soup.select('li.interlanguage-link > a')]

    if links != []:
        for language, link in links:
            if language == 'ar':
                response = http.request('GET', link)
                soup = BeautifulSoup(response.data)

                arabic_phrase = soup.title.text
                arabic_phrase = re.sub('- ويكيبيديا', '', arabic_phrase)
                return arabic_phrase, link

    return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

def remove_file_instances(s):
    if '[[' in s and ']]' in s and 'File' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    elif '[[' in s and ']]' in s and 'Image' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    else:
        return s

# def to_delete(s):
#     return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s

def to_delete(s):
    # return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s
    return 'name' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'Current' in s or 'Formerly' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s or 'cite' in s or 'citation' in s or 'Cite' in s or 'http' in s

def get_list_items(s):
    # s = re.sub('{{.*?}}', '', s)
    # print(1, s)
    s = re.sub('\'', '', s)
#     print(2, s)
    s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
#     print(3, s)
    s = re.sub('\n----', '<br>', s)
#     print(4, s)
    s = re.sub('\n\*', '<br>', s)
#     print(5, s)
    s = re.sub('<br />', '<br>', s)
#     print(6, s)
    s = re.sub('<br/>', '<br>', s)
    s = re.sub('<Br/>', '<br>', s)
#     print(7, s)
    s = re.sub('</ref>|ref||</ref>', '', s)
#     print(8, s)
    s = re.sub('</ref>', '', s)
#     print(9, s)
    s = re.sub('ref', '', s)
#     print(10, s)
#     s = re.sub('|', '', s)
    s = s.translate({ord('|'): '<br>'})
#     print(11, s)
    s = s.translate({ord('{'): None, ord('}'): None})
#     print(11, s)
#     if '<br>' in s:
    s = s.split("<br>")
#     print(12, s)
#     else:
#         s = s.split("\|")
    s = [e.strip() for e in s if e.strip() != '']
#     print(13, s)
    s = [remove_file_instances(e).strip() for e in s]
#     print(14, s)
    s = [e for e in s if not to_delete(e)]
#     print(15,  s)
    s = list(set(s)) # keep unique entries

    s = [e for e in s if not e.isdigit()] # remove entries that are just numbers
#     print('s is now: ', s)
    if len(s) > 1:
        new_s = []
        for e in s:
            if ',' in e:
                splitted_e = e.split(',')
                for ee in splitted_e:
                    new_s.append(ee)
            elif 'to' in e:
                splitted_e = e.split('to')
                for ee in splitted_e:
                    new_s.append(ee)
            else:
                new_s.append(e)
    #     print('s is now after: ', new_s)
        new_s = [e.strip() for e in new_s if e.strip() != '']
        return new_s
    else:
        return s


def jaccard(a, b):
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


In [29]:
def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [30]:
df_political_party_en = pd.DataFrame(columns=['Name', 'Ideology', 'Political Position', 'Founder(s)'])
df_political_party_ar = pd.DataFrame(columns=['الاسم','الإيديولوجيا', 'الموقف السياسي', 'المؤسس (المؤسسون)'])

In [31]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'ideology' in page.data['infobox']:
                    print(page)

                    leaders_types = [k for k in page.data['infobox'].keys() if 'leader' in k]
                    if leaders_types == []:
                        pass
                    else:
                        founders = ''
                        count = 0
                        for lt in leaders_types:
                            if count%2 == 0:
                                pass
#                                 founders += ''.join(get_list_items(page.data['infobox'][lt]))+ ':'
                            else:
                                print(page.data['infobox'][lt])
                                founders += ','.join(get_list_items(page.data['infobox'][lt])) + ','
                            count += 1

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    print(name)
                    if isinstance(name, list):
                        if len(name) > 1:
                            print(name)
                            jsims = []
                            for i, n in enumerate(name):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name = [name[jsims.index(max(jsims))]]
                            print('name after: ', name)
                    ideology = get_list_items(page.data['infobox']['ideology'])
                    position = get_list_items(page.data['infobox']['position']) if 'position' in page.data['infobox'] else '-'

                    df_political_party_en = pd.concat([df_political_party_en, pd.DataFrame({
                        'Name': [','.join(name if isinstance(name, list) else name)],
                        'Ideology': [','.join(ideology) if isinstance(ideology, list) else ideology],
                        'Political Position': [','.join(position) if isinstance(position, list) else position],
                        'Founder(s)': [founders],
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('founders: ', founders)
                    print('ideology: ', ideology)
                    print('position: ', position)



                    # get the arabic name, ideology, political position, and founders of the political party
#                     page_ar = wptools.page(p, lang='ar')
#                     page_ar.get_parse()
#                     name_ar = page_ar.data['title']

                    if name != '':
                        print('START')
                        name_ar = []
                        for n in name:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('name: ', n)
                                print('name ar: ', n_ar),
                                print('name ar link: ', n_ar_link)
                                name_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        name_ar = '-'

                    print('START')
                    ideology_ar = []
                    for ideo in ideology:
                        ideo_ar, ideo_ar_link = get_arabic_name(ideo)
                        if ideo_ar != -1:
                            print('ideology: ', ideo)
                            print('ideology ar: ', ideo_ar),
                            print('ideo ar link: ', ideo_ar_link)
                            ideology_ar.append(ideo_ar)
                        else:
                            continue
                    print('END')

                    if position != '-':
                        position_ar = []
                        print('START')
                        for pos in position:
                            pos_ar, pos_ar_link = get_arabic_name(pos)
                            if pos_ar != -1:
                                print('position: ', pos)
                                print('position ar: ', pos_ar),
                                print('position ar link: ', pos_ar_link)
                                position_ar.append(pos_ar)
                            else:
                                continue
                        print('END')
                    else:
                        position_ar = '-'

                    print('START')
                    founders_ar = []
                    for f in founders.split(','):
                        if f.strip() != '':
                            f_ar, f_ar_link = get_arabic_name(f)
                            if f_ar != -1:
                                print('founder: ', f)
                                print('founder ar: ', f_ar),
                                print('founder ar link: ', f_ar_link)
                                founders_ar.append(f_ar)
                            else:
                                continue
                    print('END')


                    df_political_party_ar = pd.concat([df_political_party_ar, pd.DataFrame({
                        'الاسم': [','.join(name_ar)],
                        'الإيديولوجيا': [','.join(ideology_ar)],
                        'الموقف السياسي': [','.join(position_ar)],
                        'المؤسس (المؤسسون)': [','.join(founders_ar)],
                    })], ignore_index=True)


        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 162nd Division (IDF)
en.wikipedia.org (imageinfo) File:Logo-ugda-162.png
en.wikipedia.org (parse) 17 May Agreement
en.wikipedia.org (parse) 1948 Arab-Israeli war
en.wikipedia.org (parse) 1948 Arab–Israeli War
en.wikipedia.org (parse) 1948 Palestine war
en.wikipedia.org (parse) 1949 Armistice Agreements
en.wikipedia.org (parse) 1968 Israeli raid on Lebanon
en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:PikiWiki Israel 4220 Israel Def...
en.wikipedia.org (parse) 1981 Iraqi embassy bombing in Beirut
en.wikipedia.org (parse) 1982 (2019 film)
en.wikipedia.org (imageinfo) File:1982 2019 poster.jpg
en.wikipedia.org (parse) 1982 Beirut bombing
en.wikipedia.org (parse) 1982 kidnapping of Iranian diplomats
en.wikipedia.org (parse) 1983 Beirut barracks bombings
en.wikipedia.org (imageinfo) File:Beirutbarr.jpg
en.wikipedia.org (parse) 1983 US embassy bombing in Beirut
en.wikipedia.org (imageinfo) File:Beirutembassy.jpg
en.wikipedia.o

['Abu Nidal Organization']
-----------------
name:  ['Abu Nidal Organization']
founders:  
ideology:  ['Pan-Arabism', 'Anti-Zionism', 'Palestinian nationalism', 'Secularism']
position:  ['Left-wing']
START
name:  Abu Nidal Organization
name ar:  حركة فتح - المجلس الثوري 
name ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B1%D9%83%D8%A9_%D9%81%D8%AA%D8%AD_-_%D8%A7%D9%84%D9%85%D8%AC%D9%84%D8%B3_%D8%A7%D9%84%D8%AB%D9%88%D8%B1%D9%8A
END
START
Cannot find the link: http://en.wikipedia.org/wiki/PanArabism
ideology:  Pan-Arabism
ideology ar:  القومية العربية (مترجمه)
ideo ar link:  -1
ideology:  Anti-Zionism
ideology ar:  معاداة الصهيونية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%A7%D8%AF%D8%A7%D8%A9_%D8%A7%D9%84%D8%B5%D9%87%D9%8A%D9%88%D9%86%D9%8A%D8%A9
ideology:  Palestinian nationalism
ideology ar:  وطنية فلسطينية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%88%D8%B7%D9%86%D9%8A%D8%A9_%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86%D9%8A%D8%A9
ideology:  Secularism
ideolo

en.wikipedia.org (parse) Abu Nidal group
en.wikipedia.org (imageinfo) File:Abu Nidal Organization flag.jpg


['Abu Nidal Organization']
-----------------
name:  ['Abu Nidal Organization']
founders:  
ideology:  ['Pan-Arabism', 'Anti-Zionism', 'Palestinian nationalism', 'Secularism']
position:  ['Left-wing']
START
name:  Abu Nidal Organization
name ar:  حركة فتح - المجلس الثوري 
name ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B1%D9%83%D8%A9_%D9%81%D8%AA%D8%AD_-_%D8%A7%D9%84%D9%85%D8%AC%D9%84%D8%B3_%D8%A7%D9%84%D8%AB%D9%88%D8%B1%D9%8A
END
START
Cannot find the link: http://en.wikipedia.org/wiki/PanArabism
ideology:  Pan-Arabism
ideology ar:  القومية العربية (مترجمه)
ideo ar link:  -1
ideology:  Anti-Zionism
ideology ar:  معاداة الصهيونية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%A7%D8%AF%D8%A7%D8%A9_%D8%A7%D9%84%D8%B5%D9%87%D9%8A%D9%88%D9%86%D9%8A%D8%A9
ideology:  Palestinian nationalism
ideology ar:  وطنية فلسطينية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%88%D8%B7%D9%86%D9%8A%D8%A9_%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86%D9%8A%D8%A9
ideology:  Secularism
ideolo

en.wikipedia.org (parse) Agreement on Disengagement between Israe...
en.wikipedia.org (parse) Ain al-Hilweh
en.wikipedia.org (parse) Ain es Saheb airstrike
en.wikipedia.org (imageinfo) File:Ein saheb strike.jpg
en.wikipedia.org (parse) Air Enthusiast
en.wikipedia.org (parse) Aishiyeh massacre
en.wikipedia.org (parse) Al-Aqsa Martyrs' Brigades
en.wikipedia.org (imageinfo) File:Al Aqsa Martyrs' Brigades Flag.gif


...

Rabie Banat
[[Antoun Saadeh]]
['Syrian Social Nationalist Party']
-----------------
name:  ['Syrian Social Nationalist Party']
founders:  Rabie Banat,Antoun Saadeh,
ideology:  ['Fascism', 'Syrian nationalism', 'Social nationalism', 'collapsible list', 'Antisemitism', 'Anti-communism', 'Economic populism', 'ubl', 'Syrian irredentism']
position:  ['Syncretic efn', 'and right', 'sometimes being labeled far-right.', 'sometimes being labeled far-right.', 'In the past', 'the party or elements of its ideology or membership have been erred to as belonging to both the political left', 'and right']
START
name:  Syrian Social Nationalist Party
name ar:  الحزب السوري القومي الاجتماعي 
name ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D8%B3%D9%88%D8%B1%D9%8A_%D8%A7%D9%84%D9%82%D9%88%D9%85%D9%8A_%D8%A7%D9%84%D8%A7%D8%AC%D8%AA%D9%85%D8%A7%D8%B9%D9%8A
END
START
ideology:  Fascism
ideology ar:  فاشية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%81%D8%A7%D8%B4

en.wikipedia.org (parse) Syrian Social Nationalist Party in Lebanon


Rabi Banat
[[Antun Saadeh]]
['Syrian Social Nationalist Party']
-----------------
name:  ['Syrian Social Nationalist Party']
founders:  Rabi Banat,Antun Saadeh,
ideology:  ['Fascism', 'Syrian nationalism', 'Social nationalism', 'Antisemitism collapsible list', 'Corporatism', 'Page needed', 'August 2021', 'Arab nationalism', 'Anti-communism Page needed', 'Pan-Syrianism', 'Pan-Arabism', 'ubl', 'Syrian irredentism']
position:  ['Syncretic']
START
name:  Syrian Social Nationalist Party
name ar:  الحزب السوري القومي الاجتماعي 
name ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D8%B3%D9%88%D8%B1%D9%8A_%D8%A7%D9%84%D9%82%D9%88%D9%85%D9%8A_%D8%A7%D9%84%D8%A7%D8%AC%D8%AA%D9%85%D8%A7%D8%B9%D9%8A
END
START
ideology:  Fascism
ideology ar:  فاشية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%81%D8%A7%D8%B4%D9%8A%D8%A9
ideology:  Syrian nationalism
ideology ar:  القومية السورية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%82%D9%88%D9%85%D9%8

en.wikipedia.org (parse) Syrian intervention in the Lebanese Civil War
en.wikipedia.org (imageinfo) File:Syrian team with Milan.jpg
en.wikipedia.org (parse) Syrian occupation of Lebanon
en.wikipedia.org (imageinfo) File:Lebanon civil war map 1976.gif
en.wikipedia.org (parse) T-34
en.wikipedia.org (imageinfo) File:Tank T-34.JPG
en.wikipedia.org (parse) T-54/55
en.wikipedia.org (imageinfo) File:T-55 Tank Outside Imperial War ...
en.wikipedia.org (parse) T-72
en.wikipedia.org (imageinfo) File:Alabino05042017-40.jpg
en.wikipedia.org (parse) Taba Summit
en.wikipedia.org (parse) Tayibe (Lebanon)
en.wikipedia.org (parse) Tel Aviv
en.wikipedia.org (parse) The Bhamdoun abduction operation (1982)
en.wikipedia.org (parse) The Clinton Parameters
en.wikipedia.org (parse) The Fateful Triangle: the United States,...
en.wikipedia.org (imageinfo) File:Noam Chomsky portrait 2017 reto...
en.wikipedia.org (parse) The Peace Treaty between Israel and Lebanon
en.wikipedia.org (parse) The Pentagon
en.wikipedi

skipping Wikipedia:Accuracy dispute
skipping Wikipedia:Avoid weasel words
skipping Wikipedia:Citation needed
skipping Wikipedia:Citing sources
skipping Wikipedia:Link rot
skipping Wikipedia:Protection policy
skipping Wikipedia:Verifiability
skipping Template:Arab–Israeli armed engagements
skipping Template:Arab–Israeli conflict
skipping Template:Arab–Israeli conflict participants
skipping Template:Arab–Israeli diplomacy
skipping Template:Campaignbox 1982 Lebanon War
skipping Template:Campaignbox Lebanese Civil War
skipping Template:Cite book
skipping Template:Country data Syria
skipping Template:Israeli wars
skipping Template talk:Arab–Israeli armed engagements
skipping Template talk:Arab–Israeli conflict
skipping Template talk:Arab–Israeli conflict participants
skipping Template talk:Arab–Israeli diplomacy
skipping Template talk:Campaignbox 1982 Lebanon War
skipping Template talk:Campaignbox Lebanese Civil War
skipping Template talk:Israeli wars
skipping Help:Authority control
skippin

In [32]:
df_political_party_en

,Name,Ideology,Political Position,Founder(s)
0,Abu Nidal Organization,"Pan-Arabism,Anti-Zionism,Palestinian nationali...",Left-wing,
1,Abu Nidal Organization,"Pan-Arabism,Anti-Zionism,Palestinian nationali...",Left-wing,
2,Al-Aqsa Martyrs Brigades,"Anti-Zionism,<!-- Palestinian nationalism and ...",-,
3,,"Anti-imperialism,Nasserism,Arab socialism,Arab...",Left-wing,
4,Al-Tanzim,"Ultranationalism,Anti-communism,Lebanese natio...",-,
...,...,...,...,...
56,South Lebanon Army,"Lebanese nationalism,Maronite politics,Anti-Pa...",-,","
57,South Lebanon Army,"Lebanese nationalism,Maronite politics,Anti-Pa...",-,","
58,Syrian Social Nationalist Party,"Fascism,Syrian nationalism,Social nationalism,...","Syncretic efn,and right,sometimes being labele...","Rabie Banat,Antoun Saadeh,"
59,Syrian Social Nationalist Party,"Fascism,Syrian nationalism,Social nationalism,...","Syncretic efn,and right,sometimes being labele...","Rabie Banat,Antoun Saadeh,"


In [33]:
df_political_party_ar

,الاسم,الإيديولوجيا,الموقف السياسي,المؤسس (المؤسسون)
0,حركة فتح - المجلس الثوري,"القومية العربية (مترجمه),معاداة الصهيونية ,وطن...",يسارية,
1,حركة فتح - المجلس الثوري,"القومية العربية (مترجمه),معاداة الصهيونية ,وطن...",يسارية,
2,كتائب شهداء الأقصى (مترجمه),"معاداة الصهيونية ,يمكن افتراض القومية الفلسطين...",-,
3,,"معاداة الاستعمارية ,التيار الناصري ,اشتراكية ع...",يسارية,
4,التنظيم (مترجمه),"عصبية قومية ,معاداة الشيوعية ,قومية لبنانية",-,
...,...,...,...,...
56,جيش لبنان الجنوبي,"قومية لبنانية ,مارونية سياسية ,معاداة فلسطين (...",-,
57,جيش لبنان الجنوبي,"قومية لبنانية ,مارونية سياسية ,معاداة فلسطين (...",-,
58,الحزب السوري القومي الاجتماعي,"فاشية ,القومية السورية ,قومية يسارية ,قائمة قا...","efn مزامنة (مترجمه),و يمين (مترجمه),في بعض الأ...","داء الكلب بنات (مترجمه),أنطون سعادة"
59,الحزب السوري القومي الاجتماعي,"فاشية ,القومية السورية ,قومية يسارية ,قائمة قا...","efn مزامنة (مترجمه),و يمين (مترجمه),في بعض الأ...","داء الكلب بنات (مترجمه),أنطون سعادة"


In [54]:
datasets_dir = 'datasets_updated/1982-Lebanon-War/'

In [55]:
mkdir(datasets_dir)
df_political_party_en.to_csv(os.path.join(datasets_dir, 'political_parties_en.csv'), index=False)
df_political_party_ar.to_csv(os.path.join(datasets_dir, 'political_parties_ar.csv'), index=False, encoding='utf-8-sig')

In [36]:
df_politician_en = pd.DataFrame(columns=['Name', 'Political Party', 'Nationality', 'Religion'])
df_politician_ar = pd.DataFrame(columns=['الدين'       , 'الجنسيه'      , 'الحزب السياسي' , 'الاسم'])

In [37]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'birth_date' in page.data['infobox']:
                    print(page)

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    native_name = get_list_items(page.data['infobox']['native_name']) if 'native_name' in page.data['infobox'] else '-'
                    nickname = get_list_items(page.data['infobox']['nickname']) if 'nickname' in page.data['infobox'] else '-'

                    religion = get_list_items(page.data['infobox']['religion']) if 'religion' in page.data['infobox'] else '-'
                    party = get_list_items(page.data['infobox']['party']) if 'party' in page.data['infobox'] else '-'
                    nationality = get_list_items(page.data['infobox']['nationality']) if 'nationality' in page.data['infobox'] else '-'

                    if name == '-':
                        name_inserted = native_name
                    else:
                        name_inserted = name
                    print(name_inserted)
                    if isinstance(name_inserted, list):
                        if len(name_inserted) > 1:
                            print(name_inserted)
                            jsims = []
                            for i, n in enumerate(name_inserted):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name_inserted = [name_inserted[jsims.index(max(jsims))]][0]
                            print('name after: ', name_inserted)
                        else:
                            if name_inserted != []:
                                print('name_inserted before: {}'.format(name_inserted))
                                # name_inserted = [name_inserted]
                                # name_inserted = ','.join(name_inserted) if isinstance(name_inserted, list) else name_inserted,
                                name_inserted = name_inserted[0]
                                print('name_inserted after: {}'.format(name_inserted))

                    df_politician_en = pd.concat([df_politician_en, pd.DataFrame({
                        'Name': [name_inserted],
                        'Political Party': [','.join(party) if isinstance(party, list) else party],
                        'Nationality': [','.join(nationality) if isinstance(nationality, list) else nationality],
                        'Religion': [','.join(religion) if isinstance(religion, list) else religion],
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('native_name: ', native_name)
                    print('nickname: ', nickname)
                    print('religion: ', religion)
                    print('party: ', party)
                    print('nationality: ', nationality)

                    # get name from wikipedia
                    # get political party from wikipedia
                    # translate nationality
                    # translate religion
                    name_ar, name_ar_link = get_arabic_name(p)
                    print('name ar: ', name_ar)

                    if party != '-':
                        print('START')
                        political_party_ar = []
                        for p in party:
                            p_ar, p_ar_link = get_arabic_name(p)
                            if p_ar != -1:
                                print('political party: ', p)
                                print('political party ar: ', p_ar),
                                print('political party ar link: ', p_ar_link)
                                political_party_ar.append(p_ar)
                            else:
                                continue
                        print('END')
                    else:
                        political_party_ar = '-'

                    if nationality != '-':
                        print('START')
                        nationality_ar = []
                        for n in nationality:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('nationality: ', n)
                                print('nationality ar: ', n_ar),
                                print('nationality ar link: ', n_ar_link)
                                nationality_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        nationality_ar = '-'

                    if religion != '-':
                        print('START')
                        religion_ar = []
                        for r in religion:
                            r_ar, r_ar_link = get_arabic_name(r)
                            if r_ar != -1:
                                print('religion: ', r)
                                print('religion ar: ', r_ar),
                                print('religion ar link: ', r_ar_link)
                                religion_ar.append(r_ar)
                            else:
                                continue
                        print('END')
                    else:
                        religion_ar = '-'

                    df_politician_ar = pd.concat([df_politician_ar, pd.DataFrame({
                        'الاسم': [','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar],
                        'الحزب السياسي': [','.join(political_party_ar) if isinstance(political_party_ar, list) else political_party_ar],
                        'الجنسيه': [','.join(nationality_ar) if isinstance(nationality_ar, list) else nationality_ar],
                        'الدين': [','.join(religion_ar) if isinstance(religion_ar, list) else religion_ar],
                    })], ignore_index=True)

                    print('name ar: ', name_ar)
                    print('religion ar: ', political_party_ar)
                    print('party ar: ', party)
                    print('nationality ar: ', nationality_ar)


        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 162nd Division (IDF)
en.wikipedia.org (imageinfo) File:Logo-ugda-162.png
en.wikipedia.org (parse) 17 May Agreement
en.wikipedia.org (parse) 1948 Arab-Israeli war
en.wikipedia.org (parse) 1948 Arab–Israeli War
en.wikipedia.org (parse) 1948 Palestine war
en.wikipedia.org (parse) 1949 Armistice Agreements
en.wikipedia.org (parse) 1968 Israeli raid on Lebanon
en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:PikiWiki Israel 4220 Israel Def...
en.wikipedia.org (parse) 1981 Iraqi embassy bombing in Beirut
en.wikipedia.org (parse) 1982 (2019 film)
en.wikipedia.org (imageinfo) File:1982 2019 poster.jpg
en.wikipedia.org (parse) 1982 Beirut bombing
en.wikipedia.org (parse) 1982 kidnapping of Iranian diplomats
en.wikipedia.org (parse) 1983 Beirut barracks bombings
en.wikipedia.org (imageinfo) File:Beirutbarr.jpg
en.wikipedia.org (parse) 1983 US embassy bombing in Beirut
en.wikipedia.org (imageinfo) File:Beirutembassy.jpg
en.wikipedia.o

['Abbas al-Musawi']
name_inserted before: ['Abbas al-Musawi']
name_inserted after: Abbas al-Musawi
-----------------
name:  ['Abbas al-Musawi']
native_name:  ['عباس الموسوي', 'nobold']
nickname:  -
religion:  -
party:  -
nationality:  ['Lebanese']
Cannot find the link: http://en.wikipedia.org/wiki/Abbas alMusawi
name ar:  عباس الموسوي (مترجمه)
START
nationality:  Lebanese
nationality ar:  لبناني (مترجمه)
nationality ar link:  -1
END
name ar:  عباس الموسوي (مترجمه)
religion ar:  -
party ar:  -
nationality ar:  ['لبناني (مترجمه)']


en.wikipedia.org (parse) Abraham Accords
en.wikipedia.org (imageinfo) File:President Trump and The First L...
en.wikipedia.org (parse) Abraham Rabinovich
en.wikipedia.org (parse) Abu Nidal
en.wikipedia.org (imageinfo) File:Abu Nidal.gif


['Abu Nidal']
name_inserted before: ['Abu Nidal']
name_inserted after: Abu Nidal
-----------------
name:  ['Abu Nidal']
native_name:  ['lang', 'ar', 'أبو نضال']
nickname:  -
religion:  -
party:  []
nationality:  ['Palestinian']
name ar:  صبري البنا 
START
END
START
nationality:  Palestinian
nationality ar:  فلسطينيون 
nationality ar link:  https://ar.wikipedia.org/wiki/%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86%D9%8A%D9%88%D9%86
END
name ar:  صبري البنا 
religion ar:  []
party ar:  []
nationality ar:  ['فلسطينيون ']


en.wikipedia.org (parse) Abu Nidal Organization
en.wikipedia.org (imageinfo) File:Abu Nidal Organization flag.jpg
en.wikipedia.org (parse) Abu Nidal group
en.wikipedia.org (imageinfo) File:Abu Nidal Organization flag.jpg
en.wikipedia.org (parse) Agreement on Disengagement between Israe...
en.wikipedia.org (parse) Ain al-Hilweh
en.wikipedia.org (parse) Ain es Saheb airstrike
en.wikipedia.org (imageinfo) File:Ein saheb strike.jpg
en.wikipedia.org (parse) Air Enthusiast
en.wikipedia.org (parse) Aishiyeh massacre
en.wikipedia.org (parse) Al-Aqsa Martyrs' Brigades
en.wikipedia.org (imageinfo) File:Al Aqsa Martyrs' Brigades Flag.gif
en.wikipedia.org (parse) Al-Mourabitoun
en.wikipedia.org (parse) Al-Tanzim
en.wikipedia.org (parse) Alexander Haig
en.wikipedia.org (imageinfo) File:General Alexander Meigs Haig, Jr.jpg


...

['Zeev Almog']
name_inserted before: ['Zeev Almog']
name_inserted after: Zeev Almog
-----------------
name:  ['Zeev Almog']
native_name:  -
nickname:  -
religion:  -
party:  -
nationality:  -
name ar:  زئيف الموغ (مترجمه)
name ar:  زئيف الموغ (مترجمه)
religion ar:  -
party ar:  -
nationality ar:  -


en.wikipedia.org (parse) Ze'ev Schiff
en.wikipedia.org (imageinfo) File:Zeev sh.jpeg


['Zeev Schiff']
name_inserted before: ['Zeev Schiff']
name_inserted after: Zeev Schiff
-----------------
name:  ['Zeev Schiff']
native_name:  ['זאב שיף']
nickname:  -
religion:  -
party:  -
nationality:  ['Israel']
name ar:  زئيف شيف 
START
nationality:  Israel
nationality ar:  إسرائيل 
nationality ar link:  https://ar.wikipedia.org/wiki/%D8%A5%D8%B3%D8%B1%D8%A7%D8%A6%D9%8A%D9%84
END
name ar:  زئيف شيف 
religion ar:  -
party ar:  -
nationality ar:  ['إسرائيل ']


en.wikipedia.org (parse) Zeev Maoz
en.wikipedia.org (parse) Zrarieh raid
en.wikipedia.org (parse) Zrariyah
en.wikipedia.org (parse) Talk:1982 Lebanon War


skipping Wikipedia:Accuracy dispute
skipping Wikipedia:Avoid weasel words
skipping Wikipedia:Citation needed
skipping Wikipedia:Citing sources
skipping Wikipedia:Link rot
skipping Wikipedia:Protection policy
skipping Wikipedia:Verifiability
skipping Template:Arab–Israeli armed engagements
skipping Template:Arab–Israeli conflict
skipping Template:Arab–Israeli conflict participants
skipping Template:Arab–Israeli diplomacy
skipping Template:Campaignbox 1982 Lebanon War
skipping Template:Campaignbox Lebanese Civil War
skipping Template:Cite book
skipping Template:Country data Syria
skipping Template:Israeli wars
skipping Template talk:Arab–Israeli armed engagements
skipping Template talk:Arab–Israeli conflict
skipping Template talk:Arab–Israeli conflict participants
skipping Template talk:Arab–Israeli diplomacy
skipping Template talk:Campaignbox 1982 Lebanon War
skipping Template talk:Campaignbox Lebanese Civil War
skipping Template talk:Israeli wars
skipping Help:Authority control
skippin

In [38]:
df_politician_en

,Name,Political Party,Nationality,Religion
0,Abbas al-Musawi,-,Lebanese,-
1,Abu Nidal,,Palestinian,-
2,Alexander Haig,Republican,-,-
3,nobold,plainlist,-,-
4,René Moawad,Independent,-,-
...,...,...,...,...
65,Yekutiel Adam,-,-,-
66,Yevgeny Primakov,"Independent,Fatherland – All Russia,Communist ...",Russian,-
67,-,-,-,-
68,Zeev Almog,-,-,-


In [39]:
df_politician_ar

,الدين,الجنسيه,الحزب السياسي,الاسم
0,-,لبناني (مترجمه),-,عباس الموسوي (مترجمه)
1,-,فلسطينيون,,صبري البنا
2,-,-,جمهوري (مترجمه),ألكسندر هيغ
3,-,-,قائمة عادية (مترجمه),أرئيل شارون
4,-,-,مستقل (توضيح),رينيه معوض
...,...,...,...,...
65,-,-,-,يكوتيئيل آدم
66,-,الروسية (مترجمه),"مستقل (توضيح) ,الوطن – كل روسيا (مترجمه),الحزب...",يفكيني بريماكوف
67,-,-,-,يوسي سريد
68,-,-,-,زئيف الموغ (مترجمه)


In [56]:
mkdir(datasets_dir)
df_politician_en.to_csv(os.path.join(datasets_dir, 'politicians_en.csv'), index=False)
df_politician_ar.to_csv(os.path.join(datasets_dir, 'politicians_ar.csv'), index=False, encoding='utf-8-sig')

In [41]:
df_locations_en = pd.DataFrame(columns=['Name', 'Subdivision type', 'Subdivision name'])
df_locations_ar = pd.DataFrame(columns=['اسم التقسيم'         , 'نوع التقسيم' , 'الاسم'])

In [42]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'government_type' in page.data['infobox'] or 'subdivision_type' in page.data['infobox']:
                    print(page)

                    if 'government_type' in page.data['infobox']:
                        name = get_list_items(page.data['infobox']['common_name']) if 'common_name' in page.data['infobox'] else  page.data['infobox']['conventional_long_name'] if 'conventional_long_name' in page.data['infobox'] else '-'
                    else:
                        name = get_list_items(page.data['infobox']['official_name']) if 'official_name' in page.data['infobox'] else page.data['infobox']['name'] if 'name' in  page.data['infobox'] else '-'

                    subdivision_type = get_list_items(page.data['infobox']['subdivision_type']) if 'subdivision_type' in page.data['infobox'] else '-'
                    subdivision_name = get_list_items(page.data['infobox']['subdivision_name']) if 'subdivision_name' in page.data['infobox'] else '-'

                    name = ','.join(name) if isinstance(name, list) and len(name) >= 1 else name
                    if isEnglish(name):
                        pass
                    else:
                        name = p

                    df_locations_en = pd.concat([df_locations_en, pd.DataFrame({
                        'Name': [name],
                        'Subdivision type': [','.join(subdivision_type) if isinstance(subdivision_type, list) else subdivision_type],
                        'Subdivision name': [','.join(subdivision_name) if isinstance(subdivision_name, list) else subdivision_name]
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('subdivision_type: ', subdivision_type)
                    print('subdivision_name: ', subdivision_name)


                    if isinstance(name, list):
                        name = name[0]
                    else:
                        pass

                    if name != '-':
                        name_ar, name_ar_link = get_arabic_name(name)
                        print('name ar: ', name_ar)

                        if subdivision_type != '-':
                            print('START')
                            subdivision_type_ar = []
                            for st in subdivision_type:
                                st_ar, st_ar_link = get_arabic_name(st)
                                if st_ar != -1:
                                    print('subdivision_type: ', st)
                                    print('subdivision_type ar: ', st_ar),
                                    print('subdivision_type ar link: ', st_ar_link)
                                    subdivision_type_ar.append(st_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_type_ar = '-'


                        if subdivision_name != '-':
                            print('START')
                            subdivision_name_ar = []
                            for sn in subdivision_name:
                                sn_ar, sn_ar_link = get_arabic_name(sn)
                                if sn_ar != -1:
                                    print('subdivision_name: ', sn)
                                    print('subdivision_name ar: ', sn_ar),
                                    print('subdivision_name ar link: ', sn_ar_link)
                                    subdivision_name_ar.append(sn_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_name_ar = '-'

                        df_locations_ar = pd.concat([df_locations_ar, pd.DataFrame({
                            'الاسم': [','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar],
                            'نوع التقسيم': [','.join(subdivision_type_ar) if isinstance(subdivision_type_ar, list) else subdivision_type_ar],
                            'اسم التقسيم': [','.join(subdivision_name_ar) if isinstance(subdivision_name_ar, list) else subdivision_name_ar],
                        })], ignore_index=True)

                    else:
                        print('skipping {} because name is not found'.format(p))

        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 162nd Division (IDF)
en.wikipedia.org (imageinfo) File:Logo-ugda-162.png
en.wikipedia.org (parse) 17 May Agreement
en.wikipedia.org (parse) 1948 Arab-Israeli war
en.wikipedia.org (parse) 1948 Arab–Israeli War
en.wikipedia.org (parse) 1948 Palestine war
en.wikipedia.org (parse) 1949 Armistice Agreements
en.wikipedia.org (parse) 1968 Israeli raid on Lebanon
en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:PikiWiki Israel 4220 Israel Def...
en.wikipedia.org (parse) 1981 Iraqi embassy bombing in Beirut
en.wikipedia.org (parse) 1982 (2019 film)
en.wikipedia.org (imageinfo) File:1982 2019 poster.jpg
en.wikipedia.org (parse) 1982 Beirut bombing
en.wikipedia.org (parse) 1982 kidnapping of Iranian diplomats
en.wikipedia.org (parse) 1983 Beirut barracks bombings
en.wikipedia.org (imageinfo) File:Beirutbarr.jpg
en.wikipedia.org (parse) 1983 US embassy bombing in Beirut
en.wikipedia.org (imageinfo) File:Beirutembassy.jpg
en.wikipedia.o

-----------------
name:  Ain al-Hilweh
subdivision_type:  ['Country']
subdivision_name:  ['Lebanon', 'Flag']
Cannot find the link: http://en.wikipedia.org/wiki/Ain alHilweh
name ar:  عين الحلوة (مترجمه)
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
subdivision_name:  Flag
subdivision_name ar:  علم (راية) 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D8%B9%D9%84%D9%85_(%D8%B1%D8%A7%D9%8A%D8%A9)
END


en.wikipedia.org (parse) Ain es Saheb airstrike
en.wikipedia.org (imageinfo) File:Ein saheb strike.jpg
en.wikipedia.org (parse) Air Enthusiast
en.wikipedia.org (parse) Aishiyeh massacre
en.wikipedia.org (parse) Al-Aqsa Martyrs' Brigades
en.wikipedia.org (imageinfo) File:Al Aqsa Martyrs' Brigades Flag.gif
en.wikipedia.org (parse) Al-Mourabitoun
en.wikipedia.org (parse) Al-Tanzim
en.wikipedia.org (parse) Alexander Haig
en.wikipedia.org (imageinfo) File:General Alexander Meigs Haig, Jr.jpg
en.wikipedia.org (parse) Aley District


-----------------
name:  Aley District
subdivision_type:  ['Country']
subdivision_name:  ['Lebanon', 'Flag']
name ar:  عاليه (لبنان) 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
subdivision_name:  Flag
subdivision_name ar:  علم (راية) 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D8%B9%D9%84%D9%85_(%D8%B1%D8%A7%D9%8A%D8%A9)
END


en.wikipedia.org (parse) Alfred A. Knopf
en.wikipedia.org (imageinfo) File:Knopf Borzoi 1920.png
en.wikipedia.org (parse) All-Palestine Government
en.wikipedia.org (parse) Amal Movement
en.wikipedia.org (parse) American trusteeship proposal for Palestine
en.wikipedia.org (parse) Anglo-French Declaration
en.wikipedia.org (parse) Annapolis Conference
en.wikipedia.org (parse) Anthony Tucker-Jones
en.wikipedia.org (parse) Arab Higher Committee
en.wikipedia.org (parse) Arab League
en.wikipedia.org (parse) Arab Liberation Army
en.wikipedia.org (parse) Arab Liberation Front
en.wikipedia.org (parse) Arab Socialist Action Party – Lebanon
en.wikipedia.org (parse) Arab Studies Quarterly
en.wikipedia.org (parse) Arabic
en.wikipedia.org (imageinfo) File:Arabic albayancalligraphy.svg
en.wikipedia.org (parse) Arab–Israeli conflict
en.wikipedia.org (parse) Arab–Israeli normalization
en.wikipedia.org (parse) Ariel Sharon
en.wikipedia.org (imageinfo) File:Ariel Sharon official portrait ...
en.wikipedia.

...

-----------------
name:  North Yemen
subdivision_type:  -
subdivision_name:  -
name ar:  اليمن الشمالي 


en.wikipedia.org (parse) Yevgeny Primakov
en.wikipedia.org (imageinfo) File:YPrimakov.jpg
en.wikipedia.org (parse) Ynetnews
en.wikipedia.org (parse) Yom Kippur War
en.wikipedia.org (imageinfo) File:Yom Kippur War Montage.png
en.wikipedia.org (parse) Yossi Sarid
en.wikipedia.org (imageinfo) File:Yossi Sarid.JPG
en.wikipedia.org (parse) Ze'ev Almog
en.wikipedia.org (imageinfo) File:Rear Admiral Ze'ev Almog.jpg
en.wikipedia.org (parse) Ze'ev Schiff
en.wikipedia.org (imageinfo) File:Zeev sh.jpeg
en.wikipedia.org (parse) Zeev Maoz
en.wikipedia.org (parse) Zrarieh raid
en.wikipedia.org (parse) Zrariyah


-----------------
name:  Zrarieh
subdivision_type:  ['Country']
subdivision_name:  ['Lebanon', 'Flag']
name ar:  الزرارية 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
subdivision_name:  Flag
subdivision_name ar:  علم (راية) 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D8%B9%D9%84%D9%85_(%D8%B1%D8%A7%D9%8A%D8%A9)
END


en.wikipedia.org (parse) Talk:1982 Lebanon War


skipping Wikipedia:Accuracy dispute
skipping Wikipedia:Avoid weasel words
skipping Wikipedia:Citation needed
skipping Wikipedia:Citing sources
skipping Wikipedia:Link rot
skipping Wikipedia:Protection policy
skipping Wikipedia:Verifiability
skipping Template:Arab–Israeli armed engagements
skipping Template:Arab–Israeli conflict
skipping Template:Arab–Israeli conflict participants
skipping Template:Arab–Israeli diplomacy
skipping Template:Campaignbox 1982 Lebanon War
skipping Template:Campaignbox Lebanese Civil War
skipping Template:Cite book
skipping Template:Country data Syria
skipping Template:Israeli wars
skipping Template talk:Arab–Israeli armed engagements
skipping Template talk:Arab–Israeli conflict
skipping Template talk:Arab–Israeli conflict participants
skipping Template talk:Arab–Israeli diplomacy
skipping Template talk:Campaignbox 1982 Lebanon War
skipping Template talk:Campaignbox Lebanese Civil War
skipping Template talk:Israeli wars
skipping Help:Authority control
skippin

In [43]:
df_locations_en

,Name,Subdivision type,Subdivision name
0,Ain al-Hilweh,Country,"Lebanon,Flag"
1,Aley District,Country,"Lebanon,Flag"
2,Ashdod,Country,ISR
3,Iraq,-,-
4,Baabda,Country,Lebanon
...,...,...,...
56,United Kingdom,-,-
57,United States,-,-
58,Yemen,-,-
59,North Yemen,-,-


In [44]:
df_locations_ar

,اسم التقسيم,نوع التقسيم,الاسم
0,"لبنان ,علم (راية)",بلد,عين الحلوة (مترجمه)
1,"لبنان ,علم (راية)",بلد,عاليه (لبنان)
2,الاستخبارات والمراقبة والرصد (مترجمه),بلد,أسدود
3,-,-,العراق
4,لبنان,بلد,بعبدا
5,"لبنان ,علم (راية)",بلد,بعلبك
6,-,-,باربادوس
7,لبنان,بلد,بيروت
8,"لبنان ,علم (راية)",بلد,قضاء الشوف
9,لبنان (مترجمه),بلد,الدامور


In [57]:
mkdir(datasets_dir)
df_locations_en.to_csv(os.path.join(datasets_dir, 'locations_en.csv'), index=False)
df_locations_ar.to_csv(os.path.join(datasets_dir, 'locations_ar.csv'), index=False, encoding='utf-8-sig')